<a href="https://colab.research.google.com/github/SanjayS059/HELPURPUP/blob/main/Helpurpup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade tensorflow

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models, Model
from datasets import load_dataset
import pandas as pd
import numpy as np

# Load the dataset from Hugging Face
dataset = load_dataset("sanjayS059/Pet_disease", split="train")

# Preprocess the dataset
def preprocess_data(example):
    image = example["image"]
    label = example["label"]
    image = tf.image.resize(image, (224, 224)) / 255.0
    return {"image": image, "label": label}  # Return a dict instead of a tuple

# Apply preprocessing
dataset = dataset.map(preprocess_data)

# Convert Hugging Face dataset to TensorFlow dataset
def to_tf_dataset(hf_dataset):
    return tf.data.Dataset.from_generator(
        lambda: ((example["image"], example["label"]) for example in hf_dataset), # Yield a tuple (image, label)
        output_signature=(
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int64)
        )
    )

tf_dataset = to_tf_dataset(dataset)
tf_dataset = tf_dataset.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Build the CNN model
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
])

# Add RNN layers
input_seq = layers.Input(shape=(224, 224, 3))
cnn_out = cnn_model(input_seq)
cnn_out = layers.Reshape((1, -1))(cnn_out)

rnn_out = layers.LSTM(64, return_sequences=True)(cnn_out)
rnn_out = layers.LSTM(32)(rnn_out)

output = layers.Dense(len(dataset.features["label"].names), activation='softmax')(rnn_out)

model = Model(inputs=input_seq, outputs=output)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(tf_dataset, epochs=10)

# Save the trained model
model.save("pet_disease_cnn_rnn_model.h5")

# Load test data or use part of the dataset for prediction
test_dataset = tf_dataset.take(100)  # Taking 100 samples from the dataset for testing

# Predict on test data
predictions = model.predict(test_dataset)

# Get actual labels
true_labels = np.concatenate([y for _, y in test_dataset], axis=0)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Prepare the DataFrame for CSV
df = pd.DataFrame({
    "True Label": true_labels,
    "Predicted Label": predicted_labels
})

# Save the DataFrame to a CSV file
df.to_csv("pet_disease_predictions.csv", index=False)

print("Predictions saved to pet_disease_predictions.csv")


Resolving data files:   0%|          | 0/386 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 163s 12s/step - accuracy: 0.2916 - loss: 1.4838
Epoch 2/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


13/13 ━━━━━━━━━━━━━━━━━━━━ 165s 12s/step - accuracy: 0.5251 - loss: 1.2136
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 167s 13s/step - accuracy: 0.5251 - loss: 1.2606
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 202s 13s/step - accuracy: 0.5251 - loss: 1.3062
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 202s 13s/step - accuracy: 0.5251 - loss: 1.3415
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 164s 12s/step - accuracy: 0.5251 - loss: 1.3671
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 166s 13s/step - accuracy: 0.0654 - loss: 1.3855
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 202s 13s/step - accuracy: 0.0654 - loss: 1.3988
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 204s 13s/step - accuracy: 0.0654 - loss: 1.4094
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 166s 13s/step - accuracy: 0.0654 - loss: 1.4133


13/13 ━━━━━━━━━━━━━━━━━━━━ 138s 10s/step
Predictions saved to pet_disease_predictions.csv


In [8]:
pip install Flask tensorflow pillow

In [ ]:
from flask import Flask, request, render_template, redirect, url_for
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

# Load the trained model
model = load_model('pet_disease_cnn_rnn_model.h5')

# Class names or labels
class_names = ['Disease A', 'Disease B', 'Disease C', ...]  # Replace with actual class names

app = Flask(__name__)

# Function to preprocess the image
def preprocess_image(image):
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Route for the homepage
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file:
            image = Image.open(file)
            processed_image = preprocess_image(image)
            prediction = model.predict(processed_image)
            predicted_label = class_names[np.argmax(prediction)]
            return render_template('result.html', label=predicted_label)
    return render_template('index.html')

# Route for the result page
@app.route('/result')
def result():
    return render_template('result.html')

if __name__ == '__main__':
    app.run(debug=True)
